# 3. 함수 정의와 호출
3장에서는 함수 선언과 호출이 코틀린에선 어떠한지 살펴본다. 이 과정에서 확장함수와 프로퍼티를 사용해 코틀린스럽게 코드를 작성하는 법도 알아본다.

## 코틀린에서 컬렉션 만들기
리스트와 맵을 만들어보자.

In [2]:
fun main() {
    val list = listOf(1, 2, 3)
    val map = mapOf(1 to "one",
        2 to "two",
        3 to "three")

    println(list.javaClass) // javaClass는 자바에서 getClass()에 해당하는 코틀린 표현임.
    println(map.javaClass) //LinkedHashMap
}

main()

class java.util.Arrays$ArrayList
class java.util.LinkedHashMap


> 참고로 여기서 `to` 는 걍 함수다. (키워드가 아니다!) `Pair` 객체를 생성하는 infix함수임.

> 코틀린 '표현'이 정확히 무엇을 말하는가? 일단 `javaClass` 는 프로퍼티가 아니다. 이건 말 그대로 '표현'이고, 코틀린 컴파일러가 내부적으로 `this::class.java` 로 치환한다.
> 즉 코틀린 컴파일러가 이해할 수 있는 문법적 요소임. 표준 함수도, 확장 프로퍼티도 아니다. 컴파일러가 AST단계에서 치환함.

위에서 `println()` 을 찍어보면 알 수 있듯이 **코틀린의 list와 map은 자바의 컬렉션을 사용하고 있다.**
하지만 자바와의 차이점으로 코틀린의 컬렉션 인터페이스는 **기본값이 읽기 전용이라는 점임.** (함수형 프로그래밍을 고려한건가?)
이런 읽기 전용 컬렉션과 이에 일대일로 대응하는 가변 인터페이스들이 따로 있다.

코틀린 컬렉션은 기본적으로 자바의 컬렉션을 그대로 쓰고 있긴 하지만, 몇가지 기능이 더 추가된 인터페이스를 갖고 있.

In [3]:
fun main() {
    val Strings = listOf("first", "second", "fourteenth")
    println(Strings.last()) //리스트의 마지막 원소를 가져온다.

    println(Strings.shuffled()) //원소를 뒤섞은 버전의 리스트를 반환한다!

    val numbers = setOf(1, 14, 2)
    println(numbers.max())
    println(numbers.sum()) //수로 이뤄진 컬렉션일 경우 합계도 구해준다!
}

main()

fourteenth
[first, fourteenth, second]
14
17


## 함수를 호출하기 쉽게 만들기
자바 컬렉션에는 기본적으로 `toString()` 구현이 포함되어 있다. 하지만 대부분 이걸 직접 오버라이딩해서 사용하거나 구아바, 아파치 커먼즈 같은 라이브러리를 사용한다.

우선 코틀린의 이점이 없는 방식으로 직접 구현해보자.

In [4]:
fun <T> joinToString(
    collection: Collection<T>,
    separator: String,
    prefix: String,
    postfix: String
): String {

    val result = StringBuilder(prefix)

    for((index, element) in collection.withIndex()) {
        if (index > 0) result.append(separator)
        result.append(element)
    }

    result.append(postfix)
    return result.toString()
}

fun main() {
    val list = listOf(1, 2, 3)
    println(joinToString(list, "; ", "(", ")"))
}

main() //(1; 2; 3)

(1; 2; 3)


위의 코드가 잘 동작하긴 하지만 함수를 사용하는 쪽을 보면 아규먼트가 네개나 되어서 복잡하다. 이걸 개선하려면 함수 시그니처를 어떻게 만들어야 할까?

### 이름붙은 인자
위처럼 타입이 전부 String인데다가, 파라미터가 4개나 되면 각 파라미터가 어떤 역할을 하는지 함수 시그니처를 직접 찾아가기 전까진 헤매기 쉽다.
그래서 코틀린은 다른 언어가 그렇듯이 이름이 있는 아규먼트를 지원한다.

In [ ]:
fun <T> joinToString(
    collection: Collection<T>,
    separator: String,
    prefix: String,
    postfix: String
): String {

    val result = StringBuilder(prefix)

    for((index, element) in collection.withIndex()) {
        if (index > 0) result.append(separator)
        result.append(element)
    }

    result.append(postfix)
    return result.toString()
}

fun main() {
    val list = listOf(1, 2, 3)
    println(joinToString(collection = list,
                         separator ="; ",
                         prefix ="(",
                         postfix =")")) //파라미터에 이름을 붙여줄 수 있다.
}

main()

### 디폴트 파라미터 값
파라미터에 디폴트 값은 왜 필요할까?
자바에서 몇 메서드들은 너무 많이 오버로딩되어있다. 이들은 하위호환성이나 API 사용자들에게 편의를 제공하기 위해 오버로딩된다.
이러면 파라미터가 반복될수록 모든 오버로딩된 함수에 동일한 설명을 달아야하는 문제나, 어떤 함수가 호출될지 모호한 경우가 분명히 생긴다.

만약 파라미터에 기본값을 지정할 수 있다면 이런 오버로딩의 상당수를 피할 수 있다.

In [6]:
fun <T> joinToString(
    collection: Collection<T>,
    separator: String = ", ",
    prefix: String = "",
    postfix: String = "" //파라미터에 기본값을 지정할 수 있다.
): String {

    val result = StringBuilder(prefix)

    for((index, element) in collection.withIndex()) {
        if (index > 0) result.append(separator)
        result.append(element)
    }

    result.append(postfix)
    return result.toString()
}

fun main() {
    val list = listOf(1, 2, 3)
    println(joinToString(collection = list)) //기본값이 있기 때문에 오버로딩 하지 않아도 된다.
}

main()

1, 2, 3


> 실수할만한 요소 하나는, 함수 파라미터에 기본값을 바꾸고 다시 재컴파일하면 그 함수를 호출하는 코드 중에 값을 지정하지 않은 모든 인자는 자동으로 그 바뀐 기본값을 적용받는다는 점임.

그런데 자바에서는 디폴트 파라미터라는 개념이 없다. 그래서 코틀린 함수를 자바에서 호출할때는 무조건 모든 인자를 명시해줘야 한다.
만약 자바에서 호출하는 횟수가 잦다면 `@JvmOverloads` 어노테이션을 사용할 수 있다.
이 친구를 사용하면 코틀린 컴파일러가 자동으로 맨 마지막 파라미터로부터, 파라미터를 하나씩 생략한 오버로딩한 자바 메서드를 추가해준다.

**그리고 생성된 각각의 오버로드 함수들은 시그니처에서 생략된 파라미터들에 대해 코틀린이 지정한 디폴트 파라미터 값을 사용한다.**

### 정적인 유틸리티 클래스 없애기: 최상위 함수와 프로퍼티
사실 (지금까지 그래왔지만) 코틀린에서는 '메서드'일 필요가 없다.

가끔은 어떤 특정 클래스 하나에 포함시키기 어려운 코드가 있을 때도 있다. 또한 어떤 연산에서는 연관된 중요한 클래스가 두개 이상일때도 있다.
이런이유로 다양한 스태틱 메서드만 모아두는 클래스가 생긴다. JDK의 `Collections` 같은 동반 클래스가 그 예다. 또는 `Util` 이란 이름이 붙은 클래스가 그 예다.
코틀린에서는 함수를 소스 파일의 최상위 수준에 위치시킬 수 있어서 이런 일이 일어나지 않아도 된다.
또한 이런 최상위 수준에 정의된 함수도 여전히 그 패키지의 멤버 함수라서 다른 패키지에서 그 함수를 임포트해서 사용하는 방식도 유지된다. 하지만 이를 위해 쓸데없는 클래스를 만들고 함수를 넣을 필요가 없다.

이런게 어떻게 가능한걸까?

JVM은 원래 클래스 안에 있는 코드만을 실행할 수 있도록 설계되어있다. 그래서 **파일의 최상위 수준에 있는 함수**를 실행하려면 컴파일 하는 동안에 **새 클래스**가 만들어져야 한다.
**코틀린 컴파일러가 생성하는 이 숨은 클래스의 이름은 코틀린 소스 파일의 이름과 대응한다.**
**즉 코틀린 파일의 모든 최상위 함수는 이 클래스의 스태틱 메서드가 된다.** 그래서 자바에서는 스태틱 메서드를 다루던 방식으로 해당 함수를 호출 할 수 있다.

참고로 코틀린 컴파일러가 만드는 클래스의 이름은 `파일 이름 + Kt` 의 형식으로 만들어진다. 이걸 바꾸고 싶다면 파일 수준 어노테이션인 `@JvmName("...")` 을 사용한다.

In [ ]:
@file: JvmName("StringFunctions")

package strings

fun joinToString() {}

//...

### 최상위 프로퍼티